In [35]:
from datetime import date
tues = date(2025,10,29)
print(repr(tues))   # repr函数返回一个python可解释的表达式
print(str(tues))

datetime.date(2025, 10, 29)
2025-10-29


In [36]:
print(tues.__repr__())   # 相当于调用repr(tues)时,在tues上调用__repr__方法
print(tues.__str__())

datetime.date(2025, 10, 29)
2025-10-29


In [37]:
class Account:
    interest = 0.02    # 类属性
    def __init__(self,account_holder):
        self.balance = 0
        self.holder = account_holder
    def deposit(self,amount):
        self.balance = self.balance + amount
        return self.balance
    def withdraw(self,amount):
        if self.balance < amount:
            return 'Insufficient funds'
        self.balance = self.balance - amount
        return self.balance

Account.__bool__ = lambda self: self.balance != 0

In [38]:
jack_account = Account('Jack')
jack_account.__bool__()   # 为Account类定义专门的判断真假值的函数

False

In [39]:
print(jack_account.balance)

0


In [40]:
spock_account = Account('Spock')
spock_account.balance = 100
print(bool(spock_account))

True


In [41]:
print(spock_account.__bool__())
print(spock_account.__bool__)

True
<bound method <lambda> of <__main__.Account object at 0x00000291813C8290>>


In [42]:
# __len__()方法
print(len('Hello World!'))
'Hello World!'.__len__()

12


12

In [43]:
# 如果序列没有提供 __bool__ 方法，那么 Python 会使用序列的长度来确定其真假值。
# 空的序列是假值，而非空序列是真值。
print(bool(''))
print(bool([]))
print(bool('Hello World!'))

False
False
True


In [44]:
# __getitem__()方法
print('Hello World!'[3])
print('Hello World!'.__getitem__(3))

l
l


In [45]:
# 给对象包含一个__call__方法,可以定义一个行为像高阶函数的类,其实例对象是一个函数
class Adder(object):  # (object)
    def __init__(self,n):   # n
        self.n = n          # self.n = n,而不是return
    def __call__(self,k):
        return self.n + k

add_three_object = Adder(3) # (3) 表示给__init__的参数
add_three_object(4)

7

In [ ]:
#======2.7.3多重表示==================
class Number:   # Number 的目的不是直接被初始化，而是作为一个不同特殊数值类的超类（superclass）提供服务
    def __add__(self,other):
        return self.add(other)
    def __mul__(self,other):
        return self.mul(other)
    
class Complex(Number):
    def add(self,other):
        return ComplexRI(self.real+other.real,self.imag+other.imag)
    def mul(self,other):
        magnitude = self.magnitude * other.magnitude
        return ComplexMA(magnitude, self.angle+other.angle)


from math import atan2
class ComplexRI(Complex):
    def __init__(self,real,imag):
        self.real = real
        self.imag = imag
    
    @property    # 让下面定义的方法可以用和调用属性值一样语法调用(即不需要使用括号)
    def magnitude(self):
        return (self.real ** 2 + self.imag ** 2)**0.5
    @property
    def angle(self):
        return atan2(self.imag,self.real)
    def __repr__(self):
        return 'ComplexRI({0:g},{1:g})'.format(self.real,self.imag)


from math import sin, cos, pi
class ComplexMA(Complex):
    def __init__(self,magnitude,angle):
        self.magnitude = magnitude
        self.angle = angle
    
    @property
    def real(self):
        return self.magnitude * cos(self.angle)
    @property
    def imag(self):
        return self.magnitude * sin(self.angle)
    def __repr__(self):
        return 'ComplexMA({0:g},{1:g}*pi)'.format(self.magnitude,self.angle/pi)



In [47]:
ri = ComplexRI(5,12)

In [48]:
print(ri.real)
print(ri.imag)
print(ri.magnitude)
print(ri.angle)
print(ri.__repr__())

5
12
13.0
1.176005207095135
ComplexRI(5,12)


In [49]:
ri.real = 9 # 修改ri的real属性值
print(ri.magnitude)  # ri的magnitude属性也自动做了相应的修改

15.0


In [50]:
ma = ComplexMA(2,pi/2)
print(ma.imag)
print(ma.real)

2.0
1.2246467991473532e-16


In [51]:
ma.angle = pi
ma.real

-2.0

In [52]:
a = ComplexRI(1,2)
b = ComplexMA(2,pi/2)
a + b 
# python实际是这样运算的:a.__add__(b)
# 然后开始从对象a所在的类,即ComplexRI中开始查找__add__方法,如果没找到就到父类中找
# 最后在Number中找到,然后return a.add(b).也就是需要对对象a调用add()方法(传入参数为对象b),由于是对对象a使用方法,因此从ComplexRI开始查找add方法
# 最后在Class中找到,并计算得出结果
# 我在这的疑惑来源有两个:一,a+b为什么开始调用__add__,原因是python的解释;
# 二,为什么在Number中的计算调用了Number子类定义的方法,原因是查找的顺序是从点表达式的对象self开始的.

ComplexRI(1,4)

In [53]:
ComplexRI(0,1) * ComplexMA(0,pi)

ComplexMA(0,1.5*pi)

这个复数的例子中通过real,imag,magnitude,angle四个属性(复数接口:任何复数都必须提供这四个属性),实现了不同表示的复数的相加和相乘.
我们不需要知道复数表示的类型和具体实现细节,但我们知道它们一定有这四个属性,就可以实现复数相加和相乘
面向对象的程序设计模式:'面向接口编程,而不是面向实现编程'

In [54]:
#========2.7.4泛型函数===============

# 定义有理数类
from math import gcd
class Rational(Number):
    def __init__(self,numer,denom):
        g = gcd(numer,denom)
        self.numer = numer // g
        self.denom = denom // g
    def __repr__(self):
        return 'Rational({0},{1})'.format(self.numer,self.denom)
    # def __str__(self):     # print的时候,如果没有定义__str__方法,则调用__repr__方法
    #     # 友好的显示   
    #     return '{}/{}'.format(self.numer, self.denom)
    def add(self,other):
        nx,dx = self.numer,self.denom
        ny,dy = other.numer,other.denom
        return Rational(nx*dy+ny*dx,dx*dy)
    def mul(self,other):
        numer = self.numer*other.numer
        denom = self.denom*other.denom
        return Rational(numer,denom)
    
print(Rational(1,2)+Rational(2,3)+Rational(1,6))
print(Rational(1,2)*Rational(2,3))


Rational(4,3)
Rational(1,3)


In [ ]:
# 类型派发:为实例对象加以归属,对归于同一类型的对象进行特定的操作,如果不属于同一类,则定义新的函数,输入不同类的对象,输出想要的结果

# 判断某对象是否属于某类或继承于某类
# 内置函数isinstance(对象,类)
c = ComplexRI(1,1)
print(isinstance(c,ComplexRI))
print(isinstance(c,ComplexMA))
print(isinstance(c,Complex))
print(isinstance(c,Number))

# 为实例对象提供类型标签type_tag:有理数:'rat',复数:'com'
Rational.type_tag = 'rat'
Complex.type_tag = 'com'

# 结合复数和有理数
def add_complex_and_rational(c,r):
    return ComplexRI(c.real+r.numer/r.denom,c.imag)

def mul_complex_and_rational(c,r):
    r_magnitude,r_angle = r.numer/r.denom,0
    if r_magnitude < 0:
        r_magnitude,r_angle = -r.numer/r.denom,pi
    return ComplexMA(c.magnitude * r_magnitude,c.angle+r_angle)

def add_rational_and_complex(r,c):
    return add_complex_and_rational(c,r)
def mul_rational_and_complex(r,c):
    return mul_complex_and_rational(c,r)


True
False
True
True


In [ ]:
# 重写Number类
class Number:
    def __add__(self,other):
        if self.type_tag == other.type_tag:
            return self.add(other)
        elif (self.type_tag,other.type_tag) in self.adders:
            return self.adders[(self.type_tag,other.type_tag)](self,other)
        
    def __mul__(self,other):
        if self.type_tag == other.type_tag:
            return self.mul(other)
        elif (self.type_tag,other.type_tag) in self.multipliers:
            return self.multipliers[(self.type_tag,other.type_tag)](self,other)

    # def cross_apply()
    adders = {('com','rat'):add_complex_and_rational,
              ('rat','com'):add_rational_and_complex}
    
    multipliers = {('com','rat'):mul_complex_and_rational,
                   ('rat','com'):mul_rational_and_complex}

# See in 2.7object abstraction-rewrite Number.ipynb

In [ ]:
#========强制转换=================
# See in 2.7object abstraction-rewrite Number-2.ipynb